SQL para streamlit ml y neural

In [ ]:
import streamlit as st
import pandas as pd
import pickle
import mysql.connector 

La parte de usar el modelo me pieerdo batante he hecho algo pero creo que no esta bien lo dejo comentado para que le echeis un vistazo

In [ ]:
# Título de la aplicación
st.title("Hosting Price Prediction and Model Comparison")

# Función para conectarse a la base de datos SQL
@st.cache_resource
def get_connection():
    return mysql.connector.connect(
        host="localhost",  
        user="root",  
        password="contraseña123",  # Cambiar por tu contraseña
        database="AIRBNB"  
    )

# Cargar el CSV de entrenamiento
@st.cache_resource
def load_training_data():
    return pd.read_csv("df_processed_to_ML.csv")

train_data = load_training_data()


# Cargar el modelo predictivo desde pickle
@st.cache_resource
def load_model():
    try:
        with open("__", "rb") as f:#No se que pkl poner porque no se cual es el del modelo 
            model = pickle.load(f)
        return model
    except FileNotFoundError:
        st.error("The predictive model file was not found.")
        return None

model = load_model()

# Sidebar: Opciones principales
st.sidebar.title("Menu Options")
menu = st.sidebar.radio(
    "Select an option:",
    ("Use a Model", 'Metrics', "Why These Metrics?"))


#Si selecciona use a model
if menu == "Use a Model Predictively":
    st.header("Use the Predictive Model to Estimate Price")
'''
    #Conectar a la base de datos
    conn = get_connection()
    cursor = conn.cursor()

    # Parámetros numéricos
    st.sidebar.subheader("Select Numeric Parameters")
    baños = st.sidebar.number_input("Baños", min_value=0, max_value=15, step=1)
    camas = st.sidebar.number_input("Camas", min_value=0, max_value=15, step=1)
    dormitorios = st.sidebar.number_input("Dormitorios", min_value=0, max_value=15, step=1)
    maximum_guests = st.sidebar.number_input("Maximum Guests", min_value=0, max_value=15, step=1)

    # Validar si los valores existen en la base de datos
    query = """
        SELECT 1 
        FROM Description_ 
        WHERE baños = %s AND camas = %s AND dormitorios = %s AND maximum_guests = %s 
        LIMIT 1;
    """
    cursor.execute(query, (baños, camas, dormitorios, maximum_guests))
    exists = cursor.fetchone()

    if not exists:
        st.warning("The selected combination of values does not exist in our database. Please choose different numbers.")
    else:
        # Parámetro categórico
        st.sidebar.subheader("Select Property Type")
        property_type = st.sidebar.radio(
            "Property Type",
            ("Entire Place", "Room")
        )

        # Codificar el tipo de propiedad
        property_type_encoded = 0 if property_type == "Entire Place" else 1

        # Calcular las medias de otras columnas relevantes , esto es un ejemplo aqui hay que poner todas las columnas usadas para el modelo
        query_avg = """
            SELECT 
                AVG(cleaning_fee), AVG(total_hours_checkin), AVG(prices_per_night) 
            FROM Description_;
        """
        cursor.execute(query_avg)
        cleaning_fee_avg, total_hours_checkin_avg, avg_price = cursor.fetchone()

        # Crear la entrada para el modelo predictivo
        user_input = pd.DataFrame({
            "baños": [baños],
            "camas": [camas],
            "dormitorios": [dormitorios],
            "maximum_guests": [maximum_guests],
            "property_type": [property_type_encoded],})

        # Realizar la predicción
        if model:
            prediction = model.predict(user_input)[0]

            # Mostrar la predicción
            st.subheader("Prediction")
            st.write(f"Based on the selected parameters, the predicted price per night is: **€{prediction:.2f}**")

'''
    
# Si selecciona Metrics
elif menu == "Metrics": #es elif porque se presupone que usar el modelo es if
    st.header("See the metrics")
    
    # Sub-opción: Elegir entre ML o Redes Neuronales
    model = st.sidebar.radio(
        "Select the type of model:",
        ("Machine Learning", "Neural Networks"))
    
    if model == "Machine Learning":
        st.subheader("Machine Learning Model")
        
        try:
            with open("resultados_modelos.pkl", "rb") as f:
                results = pickle.load(f) 
                
                if isinstance(results, pd.DataFrame):  
                    st.subheader("Overall Model Performance")
                    st.write("This table compares the performance of different models:")
                    st.dataframe(results) 
                    
                    model_names = results["model_name"].unique()
                    selected_model = st.selectbox("Select a model to view details:", model_names)                    
                    
                    model_details = results[results["model_name"] == selected_model]
                    st.subheader(f"Performance Metrics for {selected_model}")
                    st.dataframe(model_details)
            
                else:
                        st.error("The pickle file does not contain a valid DataFrame.")
        except FileNotFoundError:
                st.error("The file `resultados_modelos.pkl` was not found. Please upload it to the app directory.")
        except Exception as e:
                st.error(f"An error occurred while loading the pickle file: {e}")
        
        st.subheader("Machine Learning best hyperparameters")
        st.text("Using LGBMRegressor.")
        try:
            with open("grid_search_resultados.pkl", "rb") as f:
                ml_results = pickle.load(f)  # Carga del archivo pickle
                if isinstance(ml_results, pd.DataFrame):  # Si es un DataFrame
                    st.write("Results from the grid search:")
                    st.dataframe(ml_results)
                else:
                    st.error("The pickle file does not contain a valid DataFrame.")
        except FileNotFoundError:
            st.error("The file `grid_search.pkl` was not found. Please upload it to the app directory.")
        except Exception as e:
            st.error(f"An error occurred while loading the pickle file: {e}")
    
    #elif model == "Neural Networks":
     #   st.subheader("Neural Networks Model")
      #  st.text("Using a simple neural network built with TensorFlow/Keras.")
       # try:
        #    with open("resultados_modelos.pkl", "rb") as f:
         #       ml_results = pickle.load(f)  # Carga del archivo pickle
          #      if isinstance(ml_results, pd.DataFrame):  # Si es un DataFrame
           #         st.write("Results from the best Machine Learning model:")
            #        st.dataframe(ml_results)
             #   else:
                 #   st.error("The pickle file does not contain a valid DataFrame.")
       # except FileNotFoundError:
        #    st.error("The file `best_metrics_ml.pkl` was not found. Please upload it to the app directory.")
       # except Exception as e:
        #    st.error(f"An error occurred while loading the pickle file: {e}")

# Si selecciona "¿Por qué estas métricas?"
elif menu == "Why These Metrics?":
    st.header("Metric Explanation")
    
    # Sub-opción: Explicación de métricas para ML o Redes Neuronales
    explanation = st.sidebar.radio(
        "Select the type of model for the explanation:",
        ("Machine Learning", "Neural Networks"))
    
    if explanation == "Machine Learning":
        st.subheader("Metrics in Machine Learning")
        st.write("""
        - **MAE (Mean Absolute Error):** The average of the absolute errors between predicted and actual values.
        - **MSE (Mean Squared Error):** The average of squared errors, penalizing larger errors more heavily.
        - **R² (Coefficient of Determination):** Measures how well the model explains the variance in the data.
        """)
    
    elif explanation == "Neural Networks":
        st.subheader("Metrics in Neural Networks")
        st.write("""
        - **Loss:** The function optimized during training to improve the model.
        - **Accuracy:** The percentage of correct predictions (common in classification).
        - **MAE and MSE:** Also used in regression problems.
        """)
  
